In [1]:
import json
from os.path import abspath
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

warehouse_location = abspath('/home/jovyan/work/spark-warehouse')

print(warehouse_location)

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.debug.maxToStringFields",200) \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()


/home/jovyan/work/spark-warehouse


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/10 09:26:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
print(spark.sparkContext.getConf().getAll())

[('spark.app.startTime', '1639128394753'), ('spark.app.name', 'Python Spark SQL Hive integration example'), ('spark.executor.id', 'driver'), ('spark.driver.host', '27c3cf92ba83'), ('spark.app.id', 'local-1639128395326'), ('spark.driver.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true'), ('spark.sql.catalogImplementation', 'hive'), ('spark.rdd.compress', 'True'), ('spark.debug.maxToStringFields', '200'), ('spark.driver.port', '37597'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.sql.warehouse.dir', 'file:/home/jovyan/work/spark-warehouse'), ('spark.executor.extraJavaOptions', '-Dio.netty.tryReflectionSetAccessible=true'), ('spark.ui.showConsoleProgress', 'true')]


In [3]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
table_path='/home/jovyan/work/spark-warehouse/t_test_table'
def save(df):
    df.write\
        .mode("overwrite")\
        .partitionBy("day")\
        .parquet(table_path);
    
    
def save_by_table_name(df):
    df.write\
        .mode("overwrite")\
        .partitionBy("day")\
        .option("path",table_path)\
        .format("parquet")\
        .saveAsTable("t_test_table");

In [ ]:


df=spark.sql("select current_timestamp as time,current_date as day")
save(df)

In [5]:
df=spark.sql("select current_timestamp as time,current_date - interval -21 day as day")
save(df)
save_by_table_name(df)
spark.sql("select day,count(1) from t_test_table group by day").show(truncate=False)

+----------+--------+
|day       |count(1)|
+----------+--------+
|2021-11-29|1       |
|2021-11-27|1       |
|2021-11-30|1       |
|2021-12-10|1       |
|2021-12-08|1       |
|2021-12-09|1       |
|2021-12-14|1       |
|2021-12-12|1       |
|2021-12-11|1       |
|2021-12-31|1       |
|2021-12-20|1       |
+----------+--------+



In [4]:
spark.sql("refresh table t_test_table").show(truncate=False)
spark.sql("select day,count(1) from shopee_category_search_list_df group by day").show(truncate=False)



21/12/10 09:26:48 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/12/10 09:26:48 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/12/10 09:26:50 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
21/12/10 09:26:50 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.21.0.2
21/12/10 09:26:51 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


++
||
++
++



21/12/10 09:26:52 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
21/12/10 09:26:52 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/12/10 09:26:52 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/12/10 09:26:52 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


+----------+--------+
|day       |count(1)|
+----------+--------+
|2021-12-10|28560   |
+----------+--------+



In [ ]:
spark.catalog.listTables()